# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)
# print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = "../output_data/cities_pd.csv"
cities_pd = pd.read_csv(cities_pd, encoding="ISO-8859-1")
cities_pd_clean = cities_pd.dropna()
cities_pd_clean.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,saint-philippe,-21.36,55.77,71.60,64.0,38.0,4.70,RE,1.604616e+09
1,tasiilaq,65.61,-37.64,28.40,74.0,91.0,4.70,GL,1.604616e+09
2,kahului,20.89,-156.47,91.40,43.0,40.0,3.36,US,1.604615e+09
3,kiama,-34.68,150.87,62.01,67.0,60.0,7.00,AU,1.604616e+09
4,mataura,-46.19,168.86,55.99,76.0,46.0,4.00,NZ,1.604616e+09


In [3]:
cities_pd_clean_temp = cities_pd_clean.loc[(cities_pd_clean["Max Temp"]<90)&(cities_pd_clean["Max Temp"]>80)]
cities_pd_clean_temp.count()

City          76
Lat           76
Lon           76
Max Temp      76
Humidity      76
Cloudiness    76
Windspeed     76
Country       76
Date          76
dtype: int64

In [4]:
### Humidity Heatmap
# * Configure gmaps.
# * Use the Lat and Lng as locations and Humidity as the weight.
# * Add Heatmap layer to map.

cities_pd_clean_temp_wind = cities_pd_clean_temp.loc[cities_pd_clean_temp["Windspeed"]<10]
cities_pd_clean_temp_wind.count()

City          56
Lat           56
Lon           56
Max Temp      56
Humidity      56
Cloudiness    56
Windspeed     56
Country       56
Date          56
dtype: int64

In [6]:
cities_pd_clean_temp_wind_clouds = cities_pd_clean_temp_wind.loc[cities_pd_clean_temp_wind["Cloudiness"]<5]
cities_pd_clean_temp_wind_clouds.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
58,laguna,38.42,-121.42,81.00,24.0,1.0,2.13,US,1.604615e+09
87,saint george,37.10,-113.58,84.99,18.0,1.0,9.98,US,1.604616e+09
266,yeppoon,-23.13,150.73,80.60,39.0,0.0,4.70,AU,1.604616e+09
285,koundara,12.48,-13.30,81.00,62.0,3.0,4.16,GN,1.604616e+09
295,sarandi,-23.44,-51.87,84.20,25.0,0.0,6.93,BR,1.604616e+09


In [8]:
perfect_vacation = cities_pd_clean_temp_wind_clouds 
perfect_vacation = perfect_vacation[["Country", "City", "Max Temp", "Humidity", "Cloudiness", "Windspeed", "Lat", "Lon", "Date"]]
perfect_vacation = perfect_vacation.rename(columns = {"Lon":"Lng"})
perfect_vacation = perfect_vacation.sort_values("City")
perfect_vacation.head()

### Create new DataFrame fitting weather criteria
# * Narrow down the cities to fit weather conditions.
# * Drop any rows will null values.

,Country,City,Max Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Date
478,AU,bowen,82.4,54.0,0.0,8.05,-20.02,148.23,1.604616e+09
437,US,butte,82.0,24.0,1.0,6.93,39.65,-121.57,1.604616e+09
285,GN,koundara,81.0,62.0,3.0,4.16,12.48,-13.30,1.604616e+09
58,US,laguna,81.0,24.0,1.0,2.13,38.42,-121.42,1.604615e+09
482,AU,mackay,84.0,54.0,0.0,3.36,-21.15,149.20,1.604616e+09


In [9]:
hotel_df = perfect_vacation
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df.drop(columns=["index"])
hotel_df["Hotel Name"]= ""
hotel_df.head()

,Country,City,Max Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Date,Hotel Name
0,AU,bowen,82.4,54.0,0.0,8.05,-20.02,148.23,1.604616e+09,
1,US,butte,82.0,24.0,1.0,6.93,39.65,-121.57,1.604616e+09,
2,GN,koundara,81.0,62.0,3.0,4.16,12.48,-13.30,1.604616e+09,
3,US,laguna,81.0,24.0,1.0,2.13,38.42,-121.42,1.604615e+09,
4,AU,mackay,84.0,54.0,0.0,3.36,-21.15,149.20,1.604616e+09,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# geocoordinates
# target_coordinates = target_coordinates
target_search = "first hotel"
target_radius = 5000
target_type = "hotel"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    target_coordinate = f"{row ['Lat']} ,{row ['Lng']}"
    
#     print(target_coordinate)

# set up a parameters dictionary
    params = {
        "location": target_coordinate,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
#     print(response.url)
    hotel_name = response.json()
#     pprint(hotel_name['results'][0]['name'])
    
    try:
        hotel_value = hotel_name['results'][0]['name']
    except:    
        hotel_value = ""
#     print(hotel_value)\
    
    hotel_df.loc[index,"Hotel Name"] = hotel_value
               
hotel_df.head()

,Country,City,Max Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Date,Hotel Name
0,AU,bowen,82.4,54.0,0.0,8.05,-20.02,148.23,1.604616e+09,Horseshoe Bay Resort
1,US,butte,82.0,24.0,1.0,6.93,39.65,-121.57,1.604616e+09,
2,GN,koundara,81.0,62.0,3.0,4.16,12.48,-13.30,1.604616e+09,
3,US,laguna,81.0,24.0,1.0,2.13,38.42,-121.42,1.604615e+09,Hilton Garden Inn Sacramento Elk Grove
4,AU,mackay,84.0,54.0,0.0,3.36,-21.15,149.20,1.604616e+09,Oaks Mackay Rivermarque Hotel


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [16]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

locations_1 = cities_pd_clean[["Lat", "Lon"]]
humidity = cities_pd_clean["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations_1, weights=humidity, 
                                 dissipating=True,max_intensity=100,
                                 point_radius=10)
# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [12]:
# Add marker layer ontop of heat map


# Display figure
